In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import org.apache.spark.sql.functions._

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/01/06 14:37:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark UI

import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

import org.apache.spark.sql.functions._

In [2]:
val df = ss.read.json(EXTRACT_DIR.resolve("gene.json").toString)
  .select(
    $"ensembl_gene_id",
    $"approved_symbol",
    $"approved_name",
    $"uniprot_id".as("prim_accession"),
    explode(
      concat(array($"uniprot_id"), $"uniprot_accessions")
    ).as("uniprot_id")
  )
  // Convert empty to null to avoid ambiguity post-downstream-join
  .withColumn(
    "uniprot_id",
    when(trim($"uniprot_id") =!= "", $"uniprot_id")
    // otherwise null
  )
  .filter($"uniprot_id".isNotNull && $"ensembl_gene_id".isNotNull)
  .dropDuplicates()

json at cmd1.sc:1

16 / 16

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ensembl_gene_id: string, approved_symbol: string ... 3 more fields]

In [17]:
df.show(3, false)

show at cmd16.sc:1

16 / 16

show at cmd16.sc:1

1 / 1

+---------------+---------------+-------------------------------------+--------------+----------+
|ensembl_gene_id|approved_symbol|approved_name                        |prim_accession|uniprot_id|
+---------------+---------------+-------------------------------------+--------------+----------+
|ENSG00000213420|GPC2           |glypican 2                           |Q8N158        |Q8N158    |
|ENSG00000213420|GPC2           |glypican 2                           |Q8N158        |A4D2A7    |
|ENSG00000007545|CRAMP1         |cramped chromatin regulator homolog 1|Q96RY5        |Q96RY5    |
+---------------+---------------+-------------------------------------+--------------+----------+
only showing top 3 rows



In [23]:
// Show primary accessions with multiple genes
df
    .select($"ensembl_gene_id", $"prim_accession", $"approved_symbol", $"approved_name")
    .dropDuplicates()
    .groupBy("prim_accession")
    .agg(
        countDistinct("ensembl_gene_id").as("n_gene"), 
        collect_set($"approved_symbol").as("approved_symbol"),
        collect_set($"ensembl_gene_id").as("ensembl_gene_id"),
    )
    .sort($"n_gene".desc)
    .show(10, false)

show at cmd22.sc:11

16 / 16

show at cmd22.sc:11

1 / 1

show at cmd22.sc:11

...

show at cmd22.sc:11

1 / 1

show at cmd22.sc:11

1 / 1

+--------------+------+---------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prim_accession|n_gene|approved_symbol                                                                                    |ensembl_gene_id                                                                                                                                                                                                                               |
+--------------+------+---------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
// Pick one and dig deeper (SMN1/SMN2 in this case)
df.filter($"prim_accession" === "Q16637").show

show at cmd5.sc:1

16 / 16

show at cmd5.sc:1

1 / 1

+---------------+---------------+--------------------+--------------+----------+
|ensembl_gene_id|approved_symbol|       approved_name|prim_accession|uniprot_id|
+---------------+---------------+--------------------+--------------+----------+
|ENSG00000205571|           SMN2|survival of motor...|        Q16637|    Q16637|
|ENSG00000205571|           SMN2|survival of motor...|        Q16637|    Q549U5|
|ENSG00000205571|           SMN2|survival of motor...|        Q16637|    A8K0V4|
|ENSG00000205571|           SMN2|survival of motor...|        Q16637|    Q13119|
|ENSG00000205571|           SMN2|survival of motor...|        Q16637|    Q96J51|
|ENSG00000172062|           SMN1|survival of motor...|        Q16637|    Q16637|
+---------------+---------------+--------------------+--------------+----------+



In [25]:
// Show how many primary accessions have multiple genes
df
    .select($"ensembl_gene_id", $"prim_accession")
    .dropDuplicates()
    .groupBy("prim_accession")
    .agg((countDistinct("ensembl_gene_id").as("n_gene") > 1).as("has_multiple"))
    .groupBy("has_multiple").count
    .show(50, false)

show at cmd24.sc:7

16 / 16

show at cmd24.sc:7

1 / 1

show at cmd24.sc:7

...

show at cmd24.sc:7

1 / 1

show at cmd24.sc:7

1 / 1

+------------+-----+
|has_multiple|count|
+------------+-----+
|false       |19883|
|true        |76   |
+------------+-----+



In [24]:
// Show how many primary OR secondary accessions have multiple genes
// Note: secondary accessions should be ignored since they were often merged in the past
// (e.g. https://www.uniprot.org/uniprot/Q6LBL4)
df
    .groupBy("uniprot_id")
    .agg((countDistinct("ensembl_gene_id").as("n_gene") > 1).as("has_multiple"))
    .groupBy("has_multiple").count
    .show(50, false)

show at cmd23.sc:5

16 / 16

show at cmd23.sc:5

1 / 1

show at cmd23.sc:5

...

show at cmd23.sc:5

1 / 1

show at cmd23.sc:5

1 / 1

+------------+-----+
|has_multiple|count|
+------------+-----+
|false       |87743|
|true        |365  |
+------------+-----+



In [10]:
// Search raw data for all dupe accessions to get estimate of record counts by source
def accessions = df
    .select($"ensembl_gene_id", $"prim_accession", $"approved_symbol", $"approved_name")
    .dropDuplicates()
    .groupBy("prim_accession")
    .agg(countDistinct("ensembl_gene_id").as("n_gene"))
    .filter($"n_gene" > 1)
    .select("prim_accession").dropDuplicates().collect.map(_.getAs[String]("prim_accession")).toList
ss.read.json(EXTRACT_DIR.resolve("evidence_raw.json").toString)
    .filter(
        element_at(split($"target.id", "/"), -1).isin(accessions:_*)
    )
    .groupBy("sourceID")
    .count
    .sort($"count".desc)
    .show(50, false)

json at cmd9.sc:8

178 / 178

collect at cmd9.sc:7

16 / 16

collect at cmd9.sc:7

...

178 / 178

show at cmd9.sc:15

1 / 1

+------------------+-----+
|sourceID          |count|
+------------------+-----+
|europepmc         |22303|
|chembl            |56   |
|uniprot           |45   |
|uniprot_literature|13   |
+------------------+-----+



defined function accessions

In [4]:
// Show counts by target for the above (still all accessions)
def accessions = df
    .select($"ensembl_gene_id", $"prim_accession", $"approved_symbol", $"approved_name")
    .dropDuplicates()
    .groupBy("prim_accession")
    .agg(countDistinct("ensembl_gene_id").as("n_gene"))
    .filter($"n_gene" > 1)
    .select("prim_accession").dropDuplicates().collect.map(_.getAs[String]("prim_accession")).toList
ss.read.json(EXTRACT_DIR.resolve("evidence_raw.json").toString)
    .filter($"sourceID" =!= "europepmc")
    .filter(
        element_at(split($"target.id", "/"), -1).isin(accessions:_*)
    )
    .groupBy("sourceID", "target.id")
    .count
    .sort($"count".desc)
    .show(50, false)

json at cmd3.sc:8

178 / 178

collect at cmd3.sc:7

16 / 16

collect at cmd3.sc:7

...

178 / 178

show at cmd3.sc:16

1 / 1

+------------------+-------------------------------------+-----+
|sourceID          |id                                   |count|
+------------------+-------------------------------------+-----+
|uniprot           |http://identifiers.org/uniprot/Q16637|40   |
|chembl            |http://identifiers.org/uniprot/Q92843|40   |
|chembl            |http://identifiers.org/uniprot/P01562|11   |
|uniprot_literature|http://identifiers.org/uniprot/Q16637|9    |
|chembl            |http://identifiers.org/uniprot/P69905|5    |
|uniprot           |http://identifiers.org/uniprot/P84243|2    |
|uniprot_literature|http://identifiers.org/uniprot/P69905|2    |
|uniprot_literature|http://identifiers.org/uniprot/P84243|1    |
|uniprot_literature|http://identifiers.org/uniprot/Q9UFE4|1    |
|uniprot           |http://identifiers.org/uniprot/P68431|1    |
|uniprot           |http://identifiers.org/uniprot/Q9UFE4|1    |
|uniprot           |http://identifiers.org/uniprot/Q9NR28|1    |
+------------------+-----

defined function accessions

In [7]:
// Show raw evidence record counts for SMN{1,2}
ss.read.json(EXTRACT_DIR.resolve("evidence_raw.json").toString)
    .filter($"target.id".contains("/Q16637"))
    .groupBy("sourceID")
    .count
    .show(10, false)

json at cmd6.sc:1

178 / 178

show at cmd6.sc:7

178 / 178

show at cmd6.sc:7

1 / 1

+------------------+-----+
|sourceID          |count|
+------------------+-----+
|europepmc         |1319 |
|uniprot_literature|9    |
|uniprot           |40   |
+------------------+-----+



In [9]:
// Show raw evidence records for SMN{1,2}
ss.read.json(EXTRACT_DIR.resolve("evidence_raw.json").toString)
    .filter($"target.id".contains("/Q16637"))
    .filter($"sourceID" =!= "europepmc")
    .select("target.id", "disease.id", "sourceID", "type")
    .dropDuplicates()
    .show(50, false)

json at cmd8.sc:1

178 / 178

show at cmd8.sc:6

178 / 178

show at cmd8.sc:6

1 / 1

+-------------------------------------+--------------------------------------------+------------------+-------------------+
|id                                   |id                                          |sourceID          |type               |
+-------------------------------------+--------------------------------------------+------------------+-------------------+
|http://identifiers.org/uniprot/Q16637|http://purl.obolibrary.org/obo/MONDO_0009669|uniprot_literature|genetic_literature |
|http://identifiers.org/uniprot/Q16637|http://www.orpha.net/ORDO/Orphanet_70       |uniprot_literature|genetic_literature |
|http://identifiers.org/uniprot/Q16637|http://www.orpha.net/ORDO/Orphanet_83330    |uniprot_literature|genetic_literature |
|http://identifiers.org/uniprot/Q16637|http://purl.obolibrary.org/obo/MONDO_0009669|uniprot           |genetic_association|
|http://identifiers.org/uniprot/Q16637|http://www.orpha.net/ORDO/Orphanet_70       |uniprot           |genetic_association|
|http://